In [2]:
from collections import defaultdict

class FPNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.next = None

    def increment(self, count):
        self.count += count

In [3]:
def construct_tree(dataset, min_support):
    header_table = defaultdict(int)
    for transaction in dataset:
        for item in transaction:
            header_table[item] += 1
    header_table = {k: v for k, v in header_table.items() if v >= min_support}
    frequent_items = set(header_table.keys())
    if len(frequent_items) == 0:
        return None, None
    for k in header_table:
        header_table[k] = [header_table[k], None]

    root = FPNode(None, None, None)
    for transaction in dataset:
        transaction = [item for item in transaction if item in frequent_items]
        transaction.sort(key=lambda item: header_table[item][0], reverse=True)
        current_node = root
        for item in transaction:
            current_node = update_tree(item, current_node, header_table)
    return root, header_table

In [4]:
def update_tree(item, node, header_table):
    if item in node.children:
        node.children[item].increment(1)
    else:
        new_node = FPNode(item, 1, node)
        node.children[item] = new_node
        update_header_table(item, new_node, header_table)
    return node.children[item]

def update_header_table(item, target_node, header_table):
    if header_table[item][1] is None:
        header_table[item][1] = target_node
    else:
        current_node = header_table[item][1]
        while current_node.next is not None:
            current_node = current_node.next
        current_node.next = target_node

In [6]:
def find_frequent_itemsets(header_table, prefix, frequent_itemsets, min_support):
    for item, nodes in header_table.items():
        support = nodes[0]
        if support >= min_support:
            new_prefix = prefix.copy()
            new_prefix.add(item)
            frequent_itemsets.append(new_prefix)
            conditional_pattern_bases = []
            node = nodes[1]
            while node is not None:
                conditional_pattern_base = []
                parent = node.parent
                while parent.parent is not None:
                    conditional_pattern_base.append(parent.item)
                    parent = parent.parent
                for i in range(node.count):
                    conditional_pattern_bases.append(conditional_pattern_base)
                node = node.next
            conditional_tree, conditional_header = construct_tree(conditional_pattern_bases, min_support)
            if conditional_header is not None:
                find_frequent_itemsets(conditional_header, new_prefix, frequent_itemsets, min_support)

def fpgrowth(dataset, min_support):
    frequent_itemsets = []
    root, header_table = construct_tree(dataset, min_support)
    if root is None:
        return frequent_itemsets
    find_frequent_itemsets(header_table, set(), frequent_itemsets, min_support)
    return frequent_itemsets

In [34]:
import pandas as pd
import numpy as np 

df = pd.read_csv('Online Retail.csv')
dataset = []
i =0 
while(i < len(df)) :
    j = i
    arr = []
    while(j < len(df) and df.loc[i, 'Transaction_id'] == df.loc[j, 'Transaction_id']) :
        arr.append(df.loc[j, 'Item'])
        j = j + 1
    
    i = j
    dataset.append(arr)

print(dataset)

[['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN', 'CREAM CUPID HEARTS COAT HANGER', 'KNITTED UNION FLAG HOT WATER BOTTLE', 'RED WOOLLY HOTTIE WHITE HEART.', 'SET 7 BABUSHKA NESTING BOXES', 'GLASS STAR FROSTED T-LIGHT HOLDER'], ['HAND WARMER UNION JACK', 'HAND WARMER RED POLKA DOT'], ['ASSORTED COLOUR BIRD ORNAMENT', "POPPY'S PLAYHOUSE BEDROOM ", "POPPY'S PLAYHOUSE KITCHEN", 'FELTCRAFT PRINCESS CHARLOTTE DOLL', 'IVORY KNITTED MUG COSY ', 'BOX OF 6 ASSORTED COLOUR TEASPOONS', 'BOX OF VINTAGE JIGSAW BLOCKS ', 'BOX OF VINTAGE ALPHABET BLOCKS', 'HOME BUILDING BLOCK WORD', 'LOVE BUILDING BLOCK WORD', 'RECIPE BOX WITH METAL HEART', 'DOORMAT NEW ENGLAND'], ['JAM MAKING SET WITH JARS', 'RED COAT RACK PARIS FASHION', 'YELLOW COAT RACK PARIS FASHION', 'BLUE COAT RACK PARIS FASHION'], ['BATH BUILDING BLOCK WORD'], ['ALARM CLOCK BAKELIKE PINK', 'ALARM CLOCK BAKELIKE RED ', 'ALARM CLOCK BAKELIKE GREEN', 'PANDA AND BUNNIES STICKER SHEET', 'STARS GIFT TAPE ', 'INFLATABLE POLITICAL GLOBE '

In [35]:
frequent_items = fpgrowth(dataset, 15)
print(frequent_items)

[{'WHITE HANGING HEART T-LIGHT HOLDER'}, {'HAND WARMER SCOTTY DOG DESIGN'}, {'HAND WARMER OWL DESIGN'}, {'HAND WARMER RED RETROSPOT'}]


In [52]:
import time
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load datasets
def load_dataset(dataset_path):
    df = pd.read_csv(dataset_path)
    return df

# Encode categorical data
def encode_dataset(dataset):
    encoded = []
    i =0 
    while(i < len(dataset)) :
        j = i
        arr = []
        while(j < len(dataset) and dataset.loc[i, 'Transaction_id'] == dataset.loc[j, 'Transaction_id']) :
            arr.append(dataset.loc[j, 'Item'])
            j = j + 1
        
        i = j
        encoded.append(arr)
    
    return encoded

# Analyze FP-Growth performance
def analyze_fpgrowth_performance(datasets, min_support):
    performance_results = {}
    for dataset_name, dataset_path in datasets.items():
        print(f"Processing {dataset_name}...")
        dataset = load_dataset(dataset_path)
        # print(dataset)
        encoded_dataset = encode_dataset(dataset)
        start_time = time.time()
        frequent_itemsets = fpgrowth(encoded_dataset, min_support)
        end_time = time.time()
        execution_time = end_time - start_time
        performance_results[dataset_name] = {
            "execution_time": execution_time,
            "num_frequent_itemsets": len(frequent_itemsets)
        }
        print(f"Execution time for {dataset_name}: {execution_time} seconds")
        print(f"Number of frequent itemsets found: {len(frequent_itemsets)}")
        print("------------------------------")
    return performance_results

# Datasets to analyze
datasets = {
    "Online Retail": "Online Retail.csv",
    "Food": "Dataset.csv",
    # Add more datasets here
}

# Minimum support threshold
min_support = 7

# Analyze performance
performance_results = analyze_fpgrowth_performance(datasets, min_support)

# Display results
print("\nPerformance Results:")
for dataset_name, results in performance_results.items():
    print(f"Dataset: {dataset_name}")
    print(f"Execution Time: {results['execution_time']} seconds")
    print(f"Number of Frequent Itemsets: {results['num_frequent_itemsets']}")
    print("------------------------------")


Processing Online Retail...
Execution time for Online Retail: 0.006989717483520508 seconds
Number of frequent itemsets found: 131
------------------------------
Processing Food...
Execution time for Food: 0.0019974708557128906 seconds
Number of frequent itemsets found: 83
------------------------------

Performance Results:
Dataset: Online Retail
Execution Time: 0.006989717483520508 seconds
Number of Frequent Itemsets: 131
------------------------------
Dataset: Food
Execution Time: 0.0019974708557128906 seconds
Number of Frequent Itemsets: 83
------------------------------
